<a href="https://colab.research.google.com/github/lili-alsh/Contract_work/blob/master/Contract.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Эта часть проверяет файл выгрузки события по договорам (БИТ), все ли поля правильно заполнены. 
## Выводит список событий договоров, которые заполнены не корректно.

In [ ]:
fields = ['Дата', 'Проект договора / Договор контрагента', 'Событие договора (БИТ).Проект договора / Договор контрагента.Тип договора',
          'Событие договора (БИТ).Проект договора / Договор контрагента.Код',	'Контрагент','Событие договора (БИТ).Проект договора / Договор контрагента.Описание работ',
          'Событие договора (БИТ).Проект договора / Договор контрагента.Предмет договора',	'Событие договора (БИТ).Проект договора / Договор контрагента.Сумма договора',
          'Событие договора (БИТ).Проект договора / Договор контрагента.Валюта',	'Событие договора (БИТ).Проект договора / Договор контрагента.Подразделение-инициатор',
          'Событие договора (БИТ).Проект договора / Договор контрагента.Срок действия',	'Событие договора (БИТ).Проект договора / Договор контрагента.Вид расчетов',
          'Событие договора (БИТ).Порядок выполнения',	'Вид события', 'Событие договора (БИТ).Кол-во дней  после выполнения', 'Тип дней', 'Событие договора (БИТ).Процент',
          'Событие договора (БИТ).Сумма',	'Комментарий',	'Событие договора (БИТ).Проект договора / Договор контрагента.Стадия договора', 'Событие договора (БИТ).Дата выполнения']

In [ ]:
import pandas as pd

#Читаем файл выгрузки с договорами
data = pd.read_excel("/content/Выгрузка договоров.xlsx")

type_pmnt_to_date = ['Предоплата до конкретной даты', 'Расчет до конкретной даты']
type_pmnt_sched = ['Расчет по графику платежей']
req_adds = ['Оплата по договору', 'Спецификация / Приложение/ Заявка необходимы']
data = data[(data[fields[2]].isna()) |
            (data[fields[6]].isna()) |
            # ((data[fields[10]].isna()) & (data[fields[11]].isna())) |
            ((~data[fields[10]].isna()) & (~data[fields[11]].isna())) |
            ((~data[fields[13]].isin(type_pmnt_sched)) & (~data[fields[13]].isin(type_pmnt_to_date)) 
            & (data[fields[14]].isna())) | 
            ((data[fields[13]].isin(type_pmnt_to_date)) & (data[fields[20]].isna())) |
            ((~data[fields[14]].isna()) & (~data[fields[20]].isna())) |
            ((data[fields[16]].isna()) & (data[fields[17]].isna()) & (~data[fields[13]].isin(type_pmnt_sched))) |
            ((~data[fields[16]].isna()) & (~data[fields[17]].isna())) |
            (data[fields[18]].isna()) | (data[fields[19]].isna()) |
            (data[fields[7]] == 10) | (~data[fields[18]].isin(req_adds))]
data.to_excel('Договоры на проверку.xlsx', index=False)

# ПОСЛЕ УСТРАНЕНИЯ НЕВЕРНОГО ЗАПОЛНЕНИЯ Событий договора (БИТ) выводим список договоров с поставщиками к закрытию.

In [ ]:
import pandas as pd
from datetime import datetime

# Читаем файл выгрузки с договорами
data = pd.read_excel("/content/Выгрузка договоров.xlsx")

# Берем текущую дату
curr_date = datetime.now()
# Приводим к формату даты
date = pd.to_datetime(data[fields[10]], dayfirst=True)
data[fields[10]] = date
# Добавляем данные по незакрытым со сроком действия меньше текущей даты
data_toclose = data[(data[fields[19]] != 'Закрыт') &
            (data[fields[10]] < curr_date)]
# Убираем дубли
data_toclose = data_toclose.drop_duplicates(subset=[fields[3]])

type_cntr_suplly = ['Договор подряда или возмездного оказания услуг (расходный)', 'Договор купли-продажи (расходный)',
                    'Договор-счет, оферта (не требует подписания)', 'Договор аренды']

# Данные для закрытия договоров с поставщиками
data_supply = data_toclose[data_toclose[fields[2]].isin(type_cntr_suplly)]

# Берем данные по договорам с поставщиками, которые будем сравнивать с ОСВ
data_OSV = data[data[fields[2]].isin(type_cntr_suplly)]
data_OSV = data_OSV[(data_OSV[fields[11]] == 'До момента исполнения обязательств') & (data_OSV[fields[19]] != 'Закрыт')]
# Читаем файл выгрузки ОСВ
OSV_60 = pd.read_excel("/content/ОСВ 60.xlsx")
OSV_76 = pd.read_excel("/content/ОСВ 76.xlsx")
OSV = OSV_60.merge(OSV_76, how = 'outer')
# Объединяем файл с ОСВ
data_turn = data_OSV.merge(OSV, how = 'left', left_on=fields[3],right_on='Договоры_Код')
# Выбираем договоры, по которым нулевое сальдо на конец
data_turn = data_turn[(data_turn['Сальдо_Дебет'].isna()) & (data_turn['Сальдо_Кредит'].isna())] 

# Выбираем договоры, по которым сумма оборотов за период больше или равна сумме договора
data_turn = data_turn[(data_turn['Обороты за период_Дебет'] > data_turn[fields[7]]) | 
                      (data_turn['Обороты за период_Дебет'] == data_turn[fields[7]])]
# Убираем дубли
data_turn = data_turn.drop_duplicates(subset=[fields[3]])
# Убираем лишние столбцы
data_turn = data_turn.drop(columns=['Договоры_Код', 'Обороты за период_Дебет', 'Обороты за период_Кредит', 'Сальдо_Дебет', 'Сальдо_Кредит'])

# Объединяем договоры для закрытия по поставщикам
data_close = data_supply.merge(data_turn, how = 'outer')

# Сохраняем файл
data_close.to_excel('Договоры с поставщиками для закрытия.xlsx', index=False)

# ПОСЛЕ УСТРАНЕНИЯ НЕВЕРНОГО ЗАПОЛНЕНИЯ Событий договора (БИТ) выводим список договоров с покупателями к закрытию.

In [ ]:
import pandas as pd
from datetime import datetime

# Читаем файл выгрузки с договорами
data = pd.read_excel("/content/Выгрузка договоров.xlsx")

# Берем текущую дату
curr_date = datetime.now()
# Приводим к формату даты
date = pd.to_datetime(data[fields[10]], dayfirst=True)
data[fields[10]] = date
# Добавляем данные по незакрытым со сроком действия меньше текущей даты
data_toclose = data[(data[fields[19]] != 'Закрыт') &
            (data[fields[10]] < curr_date)]
# Убираем дубли
data_toclose = data_toclose.drop_duplicates(subset=[fields[3]])

type_cntr_suplly = ['Договор подряда или возмездного оказания услуг (расходный)', 'Договор купли-продажи (расходный)',
                    'Договор-счет, оферта (не требует подписания)', 'Договор аренды']

# Данные для закрытия договоров с покупателями
data_buy = data_toclose[~data_toclose[fields[2]].isin(type_cntr_suplly)]

# Берем данные по договорам с покупателями, которые будем сравнивать с ОСВ
data_OSV = data[~data[fields[2]].isin(type_cntr_suplly)]
data_OSV = data_OSV[(data_OSV[fields[11]] == 'До момента исполнения обязательств') & (data_OSV[fields[19]] != 'Закрыт')]
# Читаем файл выгрузки ОСВ
OSV_62 = pd.read_excel("/content/ОСВ 62.xlsx")
OSV_76 = pd.read_excel("/content/ОСВ 76.xlsx")
OSV = OSV_62.merge(OSV_76, how = 'outer')
# Объединяем файл с ОСВ
data_turn = data_OSV.merge(OSV, how = 'left', left_on=fields[3],right_on='Договоры_Код')
# Выбираем договоры, по которым нулевое сальдо на конец
data_turn = data_turn[(data_turn['Сальдо_Дебет'].isna()) & (data_turn['Сальдо_Кредит'].isna())] 

# Выбираем договоры, по которым сумма оборотов за период больше или равна сумме договора
data_turn = data_turn[(data_turn['Обороты за период_Дебет'] > data_turn[fields[7]]) | 
                      (data_turn['Обороты за период_Дебет'] == data_turn[fields[7]])]
# Убираем дубли
data_turn = data_turn.drop_duplicates(subset=[fields[3]])
# Убираем лишние столбцы
data_turn = data_turn.drop(columns=['Договоры_Код', 'Обороты за период_Дебет', 'Обороты за период_Кредит', 'Сальдо_Дебет', 'Сальдо_Кредит'])

# Объединяем договоры для закрытия по покупателям
data_close = data_buy.merge(data_turn, how = 'outer')

# Сохраняем файл
data_close.to_excel('Договоры с покупателями для закрытия.xlsx', index=False)

# ПОСЛЕ УСТРАНЕНИЯ НЕВЕРНОГО ЗАПОЛНЕНИЯ Событий договора (БИТ) выводим список договоров с прочими контрагентами дл закрытия

In [ ]:
import pandas as pd
from datetime import datetime

# Читаем файл выгрузки с договорами
data = pd.read_excel("/content/Выгрузка всех договоров.xlsx")

data = data[(data['Вид договора']=='Прочее') &
            (~data['Срок действия'].isna())]


# Берем текущую дату
curr_date = datetime.now()
# Приводим к формату даты
date = pd.to_datetime(data['Срок действия'], dayfirst=True)
data['Срок действия'] = date

# Добавляем данные по незакрытым со сроком действия меньше текущей даты
data_close = data[(data['Стадия договора'] != 'Закрыт') &
            (data['Срок действия'] < curr_date)]

# Сохраняем файл
data_close.to_excel('Договоры с прочими контрагентами для закрытия.xlsx', index=False)

# ПОСЛЕ ЗАКРЫТИЯ договоров обновляем реестр поставщиков

In [ ]:
import pandas as pd

# Читаем файл выгрузки с договорами
data = pd.read_excel("/content/Выгрузка договоров.xlsx")
inn = pd.read_excel("/content/Выгрузка всех договоров.xlsx")

type_cntr_suplly = ['Договор подряда или возмездного оказания услуг (расходный)', 'Договор купли-продажи (расходный)',
                    'Договор-счет, оферта (не требует подписания)']
exc_cntr = ['24/05/19 от 24.05.2019 поставка товаров','КУ1-001623 от 19.07.2018 поставка товаров', 'IH19052302-BTC от 10.04.2019 поставка кофе в капсулах и аксессуаров',
            '37/2018 от 03.04.2018 поставка товаров (робототехника)', '17-00318893 от 18.12.2017 сублицензионный','12-03/17 ау от 12.03.2017',
            '1 от 06.06.2018  услуги перевозки пассажиров (детей) ВРО 2018','7043-ЭО/14 от 23.07.2018 лицензионный договор']

# Отбираем действующие договоры с поставщиками
data = data[(data[fields[2]].isin(type_cntr_suplly)) &
            (data[fields[19]] == 'Исполнение') & 
            (((data[fields[18]] == 'Оплата по договору') & (data[fields[7]].isna())) | 
            (data[fields[18]] == 'Спецификация / Приложение/ Заявка необходимы')) |
            (data['Проект договора / Договор контрагента'].isin(exc_cntr))]

# Убираем дубли
data = data.drop_duplicates(subset=[fields[3]])

# Читаем файл выгрузки реестра поставщиков
reg = pd.read_excel("/content/Реестр поставщиков.xlsx")

# Обновляем срок действия в файле
new_reg = reg.merge(data, how = 'outer', left_on='Договор',right_on='Проект договора / Договор контрагента')

new_reg = new_reg[(~new_reg[fields[10]].isna()) | (~new_reg[fields[11]].isna())]
new_reg['Срок действия'] = new_reg[[fields[10], fields[11]]].fillna('').sum(axis=1)
upd = new_reg[~new_reg['ИНН'].isna()]
upd = upd.rename(columns={'Контрагент_x':'Контрагент'})
upd = upd[['Контрагент', 'ИНН', 'Плательщик НДС','Договор', 'Вид товаров / услуг / работ', 'История произведенных закупок','Срок действия',
           'Ответственное подразделение', 'Ссылка на договор']]

# Делаем данные по контрагенту и ИНН
inn = inn[['Контрагент', 'ИНН']]
inn = inn.drop_duplicates(subset=['ИНН'])

# Заводим новые данные для реестра
n_reg = new_reg[new_reg['ИНН'].isna()]
n_reg = n_reg.drop_duplicates(subset=[fields[3]])
n_reg
n_reg = n_reg[['Контрагент_y','ИНН', 'Плательщик НДС', fields[1], fields[5], fields[6], 
               'Срок действия', 'Ответственное подразделение', 'Ссылка на договор']]
new = n_reg.merge(inn, how = 'left', left_on='Контрагент_y', right_on='Контрагент')
new = new[['Контрагент_y', 'ИНН_y', 'Плательщик НДС', fields[1], fields[5], fields[6], 
               'Срок действия', 'Ответственное подразделение', 'Ссылка на договор']]
new = new.rename(columns={'Контрагент_y':'Контрагент', 'ИНН_y':'ИНН','Проект договора / Договор контрагента':'Договор', fields[5]:'Описание работ',
                         fields[6]:'Предмет договора','Ответственное подразделение':'Подразделение-инициатор'})

# Убираем лишние договоры
exc_list = ['Договор на разработку ПО № 1-MVR от 01.06.2020']
new = new[((new['Контрагент'] != 'ИННОПОЛИС АО') &
          (new['Контрагент'] != 'Консультативно-диагностический центр Авиастроительного района ООО') &
          (new['Контрагент'] != 'СТАТУС ООО')) &
          (~new ['Договор'].isin(exc_list)) &
          ((~new['Договор'].str.contains('Камаз')) & 
          (~new['Договор'].str.contains('ЦНТИ')) &
          (~new['Договор'].str.contains('Конвертоплан')) &
          (~new['Договор'].str.contains('ЦК')) &
          (~new['Договор'].str.contains('БС'))) &
          (new['Договор'].str.contains('2021')) &
          (~new['Договор'].str.contains('ЛИЦ')) &
          (~new['Договор'].str.contains('ООЦ')) &
          (~new['Предмет договора'].str.contains('ЦНТИ')) &
          (~new['Предмет договора'].str.contains('ИГК')) &
          (~new['Предмет договора'].str.contains('Минпром')) &
          (~new['Предмет договора'].str.contains('Гибдд')) &
          (~new['Предмет договора'].str.contains('Технологии компонентов робототехники и мехатроники'))&
          (~new['Предмет договора'].str.contains('КЛИК'))&
          (~new['Предмет договора'].str.contains('CDO'))]

new = new.rename(columns={'Описание работ':'Вид товаров / услуг / работ', 'Предмет договора':'История произведенных закупок','Подразделение-инициатор':'Ответственное подразделение'})
new = new[['Контрагент', 'ИНН', 'Плательщик НДС', 'Договор','Вид товаров / услуг / работ','История произведенных закупок', 'Срок действия', 'Ответственное подразделение']]
# Объединить 2 датафрейма
fin_reg = pd.concat([upd, new])
fin_reg.to_excel('Для реестра поставщиков.xlsx', index=False)

# ПОСЛЕ ЗАКРЫТИЯ договоров обновляем реестр покупателей

In [ ]:
import pandas as pd
import datetime

# Читаем файл выгрузки с договорами
data = pd.read_excel("/content/Выгрузка договоров.xlsx")
data_inn = pd.read_excel("/content/Выгрузка всех договоров.xlsx")

type_cntr = ['Оказание услуг внешним контрагентам (доходный)']

# Отбираем действующие договоры с покупателями
data = data[(data[fields[2]].isin(type_cntr)) &
            (data[fields[19]] == 'Исполнение')]

# Убираем дубли
data = data.drop_duplicates(subset=[fields[3]])

# Читаем файл выгрузки реестра поставщиков
cust = pd.read_excel("/content/Реестр покупателей.xlsx")

# Обновляем срок действия в файле
upd_cust = cust.merge(data, how = 'outer', left_on='Договор',right_on='Проект договора / Договор контрагента')

upd_cust = upd_cust[(~upd_cust['Контрагент_y'].isna())&(~upd_cust['Контрагент_x'].isna())]
upd_cust = upd_cust[(~upd_cust[fields[10]].isna()) | (~upd_cust[fields[11]].isna())]
upd_cust['Срок действия'] = upd_cust[[fields[10], fields[11]]].fillna('').sum(axis=1)
upd_cust = upd_cust.rename(columns={'Контрагент_x':'Контрагент'})
upd_cust = upd_cust[['Контрагент','ИНН','Договор', 'Вид товаров / услуг / работ', 'Предмет договора', 'Срок действия', 'Ссылка на договор']]

# Делаем данные по контрагенту и ИНН
inn = data_inn[['Контрагент', 'ИНН']]
inn = inn.drop_duplicates(subset=['ИНН'])

# Отбираем для нового реестра покупателей
data['Срок действия'] = data[[fields[10], fields[11]]].fillna('').sum(axis=1)
data = data[['Контрагент', fields[1], fields[5], fields[6], 'Срок действия']]
data = data.rename(columns={fields[1]:'Договор', fields[5]:'Вид товаров / услуг / работ', fields[6]:'Предмет договора'})

new = data.merge(inn, how = 'left', left_on='Контрагент', right_on='Контрагент')
new = new[['Контрагент','ИНН','Договор', 'Вид товаров / услуг / работ', 'Предмет договора', 'Срок действия']]

# Убираем лишние договоры
new = new[(new['Контрагент'] != 'ИННОПОЛИС АО') &
          (new['Контрагент'] != 'ПЖКХ ООО') &
          (~new['Предмет договора'].str.contains('размещ'))&
          (~new['Договор'].str.contains('размещ'))&
          (~new['Предмет договора'].str.contains('аренд'))&
          (~new['Договор'].str.contains('аренд'))&
          (~new['Предмет договора'].str.contains('@innopolis.ru'))&
          (~new['Договор'].str.contains('30/17 от 21.07.2017 услуги по организации, проведению и обслуживанию мероприятий'))&
          (~new['Договор'].str.contains('Соглашение о спонсировании мероприятия от 24.04.2019'))]

# Объединить 2 датафрейма
fin_reg = pd.concat([upd_cust, new])

# Добавление соглашений
type_cntr_aggr = ['Меморандум о взаимопонимании или соглашение об академическом сотрудничестве',
                  'Соглашение о сотрудничестве с индустриальным партнером с целью поддержки студенческих проектов',
                  'Соглашение о сотрудничестве с индустриальным партнером с целью проведения совместных научно-исследовательских работ']
aggr = data_inn[data_inn['Тип договора'].isin(type_cntr_aggr)]
aggr['Срок действия_2'] = aggr[['Срок действия', 'Вид расчетов']].fillna('').sum(axis=1)
aggr['Срок действия'] = pd.to_datetime(aggr['Срок действия'])
aggr_toclose = aggr[aggr['Срок действия'] < datetime.datetime.today()]
aggr_toclose.to_excel('Соглашения для закрытия.xlsx',index=False)
aggr = aggr[(~aggr['Наименование'].isin(aggr_toclose['Наименование'])) &
            (aggr['Стадия договора'] == 'Исполнение')]
aggr = aggr[['Контрагент', 'ИНН', 'Наименование', 'Описание работ', 'Предмет договора', 'Срок действия_2']]
aggr = aggr.rename(columns={'Наименование':'Договор', 'Описание работ':'Вид товаров / услуг / работ', 'Срок действия_2':'Срок действия'})

# Объединить 2 датафрейма
fin = pd.concat([fin_reg, aggr])
fin.to_excel('Для реестра покупателей.xlsx', index=False)